In [1]:
import pandas as pd
from tqdm import tqdm
pd.set_option('max_colwidth', 10000)
pd.options.mode.chained_assignment = None
df_dev = pd.read_csv("../data/MedQA_dev.csv", index_col=0)
df_train = pd.read_csv("../data/MedQA_train.csv", index_col=0)


In [13]:
#using Azure endpoint:
import openai
import os

openai.api_type = "azure"
openai.api_base = os.getenv("OPENAI_API_BASE")
openai.api_version = "2023-07-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")
from tech_train_functions import call_open_ai
client = None

In [2]:
#using Rachel's openai:
from openai import OpenAI
client = OpenAI()
from tech_train_functions import call_open_ai

Let's focus on question 3 which we weren't able to solve so far, and call it the hard study case:

In [17]:
hard_study_case = df_dev.iloc[4]
pd.DataFrame(hard_study_case)

,4
question,"A 19-year-old boy presents with confusion and the inability to speak properly. The patient's mother says that, a few hours ago, she noticed a change in the way he talked and that he appeared to be in a daze. He then lost consciousness, and she managed to get him to the hospital. She is also concerned about the weight he has lost over the past few months. His blood pressure is 80/55 mm Hg, pulse is 115/min, temperature is 37.2°C (98.9°F), and respiratory rate is 18/min. On physical examination, the patient is taking rapid, deep breaths, and his breath has a fruity odor. Dry mucous membranes and dry skin are noticeable. He is unable to cooperate for a mental status examination. Results of his arterial blood gas analysis are shown.\nPco2 16 mm Hg\nHCO3– 10 mEq/L\nPo2 91 mm Hg\npH 7.1\nHis glucose level is 450 mg/dL, and his potassium level is 4.1 mEq/L. Which of the following should be treated first in this patient?"
answer,Hypoperfusion
answer_idx,A
options,"{'A': 'Hypoperfusion', 'B': 'Hyperglycemia', 'C': 'Metabolic acidosis', 'D': 'Hypokalemia'}"
meta_info,step1


Let's try using BM25 search to find the most similar question to this in the test split.
First let's create BM25 search DB:

In [15]:
#creating BM25 search DB:
from rank_bm25 import BM25Okapi
import numpy as np

questions_corpus=df_train['question'].tolist()
tokenized_question_corpus = [doc.split(" ") for doc in questions_corpus]
bm25_questions = BM25Okapi(tokenized_question_corpus)

def search_similar(query, bm25_index):
    query=query.lower()
    tokenized_query = query.split(" ")
    doc_scores = bm25_index.get_scores(tokenized_query)
    index=np.argsort(-doc_scores)[:5][0]
    return df_train.iloc[index]


Done! let's search for closest question to our example:

In [18]:
most_similar_in_question = search_similar(hard_study_case["question"], bm25_questions)
pd.DataFrame(most_similar_in_question)

,7116
question,"A 19-year-old man presents to the emergency department after a motor vehicle accident. The patient reports left shoulder pain that worsens with deep inspiration. Medical history is significant for a recent diagnosis of infectious mononucleosis. His temperature is 99°F (37.2°C), blood pressure is 80/55 mmHg, pulse is 115/min, and respiratory rate is 22/min. On physical exam, there is abdominal guarding, abdominal tenderness in the left upper quadrant, and rebound tenderness. The patient’s mucous membranes are dry and skin turgor is reduced. Which of the following most likely represents the acute changes in renal plasma flow (RPF) and glomerular filtration rate (GFR) in this patient?"
answer,Decreased RPF and no change in GFR
answer_idx,A
options,"{'A': 'Decreased RPF and no change in GFR', 'B': 'No change in RPF and decreased GFR', 'C': 'No change in RPF and increased GFR', 'D': 'No change in RPF and GFR'}"
meta_info,step1


What if we used the answer options only for the search db?

In [19]:
options_corpus=df_train['options'].astype(str).tolist()
tokenized_options_corpus = [doc.split(" ") for doc in options_corpus]
bm25_options = BM25Okapi(tokenized_options_corpus)

In [20]:
most_similar_in_answer = search_similar(str(hard_study_case["options"]), bm25_options)
pd.DataFrame(most_similar_in_answer)

,6718
question,"A 67-year-old man presents to his primary care physician because of weak urine stream, and increasing difficulty in initiating and stopping urination. He also reports of mild generalized body aches and weakness during the day. The past medical history includes diabetes mellitus type 2 for 35 years and essential hypertension for 19 years. The medication list includes metformin, vildagliptin, and enalapril. The vital signs include: temperature 36.7°C (98.1°F), blood pressure 151/82 mm Hg, and pulse 88/min. The physical examination is remarkable for markedly enlarged, firm prostate without nodules. The laboratory test results are as follows:\nSerum sodium 142 mEq/L\nSerum potassium 5.7 mEq/L\nSerum chloride 115 mEq/L\nSerum bicarbonate 17 mEq/L\nSerum creatinine 0.9 mg/dL\nArterial pH 7.31\n Urine pH 5.3\nUrine sodium 59 mEq/L\nUrine potassium 6.2 mEq/L\nUrine chloride 65 mEq/L\nWhich of the following most likely explains the patient’s findings?"
answer,Type 4 renal tubular acidosis
answer_idx,B
options,"{'A': 'Type 1 renal tubular acidosis', 'B': 'Type 4 renal tubular acidosis', 'C': 'Type 2 renal tubular acidosis', 'D': 'Fanconi syndrome'}"
meta_info,step2&3


Let's see if these examples are better than a random one!
First let's create the CoT example:

In [23]:
from tech_train_functions import create_example_CoT
question,answer,options = question,answer,options = most_similar_in_answer["question"],most_similar_in_answer["answer"],most_similar_in_answer["options"]
["question"],most_similar_in_answer["answer"],most_similar_in_question["options"]
system_message_explainer = "Please explain step by step how to answer this question.\
Start with 'let's take it step by step' and end with 'the correct answer is:' and then the correct answer idx and the correct answer"
example = create_example_CoT(system_message_explainer,question,options,answer, client=client)
print(example[1])

Let's take it step by step to answer this question:

1. Read the question carefully and understand the patient's presentation. The 67-year-old man is experiencing weak urine stream, difficulty in initiating and stopping urination, mild generalized body aches, and weakness during the day. He has a past medical history of diabetes mellitus type 2 and essential hypertension.

2. Look at the vital signs and laboratory test results. The vital signs include a normal temperature, elevated blood pressure, and a slightly elevated pulse. The laboratory test results show normal serum sodium and creatinine levels, elevated serum potassium, low serum bicarbonate, and an acidic arterial pH. The urine pH is also acidic, and there are elevated levels of urine sodium, potassium, and chloride.

3. Analyze the findings. The patient's symptoms, along with the laboratory test results, suggest a renal tubular acidosis (RTA). RTA is a condition where the kidneys are unable to effectively excrete acid, leadin

In [24]:
system_message_CoT = "You're a medical expert answering medical questions. Please answer step by step. Start with 'let's take it step by step'\
and end with 'the correct answer is:' and then the correct answer. Please allways end with this phrase"

q = hard_study_case["question"] +" "+ str(hard_study_case["options"])
call_open_ai(system_message_CoT, q, examples=[example], client=client)


"Let's take it step by step to answer this question:\n\n1. Read the question carefully and understand the patient's presentation. The 19-year-old boy is presenting with confusion, inability to speak properly, weight loss, and a change in his level of consciousness. His vital signs show low blood pressure, elevated pulse, and normal temperature and respiratory rate. Physical examination reveals rapid, deep breaths with a fruity odor, dry mucous membranes, and dry skin.\n\n2. Look at the arterial blood gas analysis results. The results show a low bicarbonate level (HCO3-) of 10 mEq/L, a low partial pressure of carbon dioxide (Pco2) of 16 mm Hg, and a low pH of 7.1. These findings indicate metabolic acidosis.\n\n3. Analyze the findings. The patient's symptoms, along with the arterial blood gas analysis results, suggest diabetic ketoacidosis (DKA). DKA is a life-threatening complication of uncontrolled diabetes characterized by hyperglycemia, metabolic acidosis, and ketone production.\n\n4